# 第十六章 类和函数
## 16.1 时间
编写一个print_time的函数，接收一个Time对象作为形参并以“hour：minute：second”的格式打印。<br>
提示：格式序列'%.2d'可以以最少两个字符打印一个整数，如果需要，会在前面添加前缀0。<br>

上述提示用的是旧式字符串格式化方法，应采用新的两种字符串格式方法：**1.f-string 2.str.format**<br>
旧式字符串格式化方法 ：https://docs.python.org/zh-cn/3/library/stdtypes.html#printf-style-string-formatting
1. f-string：https://docs.python.org/zh-cn/3/reference/lexical_analysis.html#f-strings
2. str.format:https://docs.python.org/zh-cn/3/library/stdtypes.html#str.format

In [11]:
class Time:
    def __init__(self,hour,minute,second):
        self.hour =  hour
        self.minute = minute
        self.second = second
        
    def print_time(self):
        print('{:02}:{:02}:{:02}'.format(self.hour,self.minute,self.second))
        
    def time_to_int(self):
        minutes = self.hour*60 + self.minute
        seconds = minutes*60 + self.second
        return seconds
    
    def is_after(self,other):
        return self.time_to_int() > other.time_to_int()

In [12]:
time1 = Time(1,1,1)
time1.print_time()

01:01:01


In [13]:
time2 = Time(10,10,10)
time2.print_time()

10:10:10


编写一个布尔函数is_after，接收两个Time对象。t1和t2，若t1在t2时间之后返则返回True，否则返回False。挑战：不用if表达式

In [14]:
time1.is_after(time2)

False

## 16.2 纯函数
编写两个用来增加时间值的函数。<br>
展示两种不同类型的函数：**纯函数**和**修改器**。<br>
展示称为**原型**和**补丁**的开发计划。

下面是add_time的一个简单原型:<br>
这个函数创建一个新的Time对象，并返回这个新对象的一个引用。这被称为一个**纯函数**。<br>
除了返回一个值之外，并不修改作为实参传入的任何对象，也没有任何如显示值或获得用户输入之类的副作用。

In [15]:
def add_time(t1,t2):
    sum = Time(t1.hour+t2.hour , t1.minute+t2.minute , t1.second+t2.second)
    return sum

创建两个time对象：<br>
start存放一个电影的开始时间<br>
duration存放电影的播放时间<br>

add_time计算出电影何时结束

In [16]:
start = Time(9,45,0)
duration = Time(1,35,0)
done = add_time(start,duration)
done.print_time()

10:80:00


结果10:80:00中分钟超过了60，所以我们需要改进add_time函数。

In [19]:
def add_time(t1,t2):
    sum = Time(0,0,0)
    
    sum.second = t1.second + t2.second
    sum.minute = t1.minute + t2.minute 
    sum.hour = t1.hour + t2.hour 
    
    if sum.second >= 60:
        sum.second-=60
        sum.miunte+=1
    if sum.minute >= 60:
        sum.minute-=60
        sum.hour+=1
        
    return sum

In [20]:
start = Time(9,45,0)
duration = Time(1,35,0)
done = add_time(start,duration)
done.print_time()

11:20:00
